In [1]:
import pandas as pd

# Load the excel file
df = pd.read_excel(r'C:\Users\PESU-RF\Downloads\Onions\Onions\raw_data\Festival dates.xlsx')

# Save it as a CSV
df.to_csv('festival_dates.csv', index=False)

In [2]:
import pandas as pd
import re

# Load the data, skipping the first descriptive header row
df = pd.read_csv('festival_dates.csv', header=1)

def parse_dates(row):
    year = str(row['Year']).strip()
    date_range_str = str(row['Approximate Date Range (DD/MM)']).strip()
    
    # Split the range by common delimiters (dash, en-dash, em-dash)
    parts = re.split(r'[–—\-]', date_range_str)
    parts = [p.strip() for p in parts]
    
    start_str = parts[0]
    # If no range is specified, end date is the same as start date
    end_str = parts[1] if len(parts) > 1 else start_str
    
    def to_iso(date_part, year_val):
        try:
            day, month = date_part.split('/')
            return f"{year_val}-{month.zfill(2)}-{day.zfill(2)}"
        except:
            return None

    start_date = to_iso(start_str, year)
    
    # Handle year rollover (e.g., Dec 20 to Jan 02)
    end_date = None
    if end_str:
        try:
            start_month = int(start_str.split('/')[1])
            end_month = int(end_str.split('/')[1])
            end_year = int(year)
            if end_month < start_month:
                end_year += 1
            end_date = to_iso(end_str, str(end_year))
        except:
            end_date = start_date
            
    return pd.Series([start_date, end_date])

# Apply date transformation
df[['date', 'date_range']] = df.apply(parse_dates, axis=1)

# Format the final DataFrame
df = df.rename(columns={'Festival / Event': 'festival name'})
final_df = df[['date', 'festival name', 'date_range']]

# Save the output
final_df.to_csv('converted_festivals.csv', index=False)